In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_box_scores(date, team1, team2):
    url_date = pd.to_datetime(date).strftime('%Y%m%d')
    url = f"https://www.basketball-reference.com/boxscores/?month={date[5:7]}&day={date[8:10]}&year={date[:4]}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    links = [a['href'] for a in soup.select('a[href^="/boxscores/"]') if 'html' in a['href']]
    game_link = None
    for link in links:
        if team1 in link and team2 in link:
            game_link = link
            break
    if not game_link:
        return pd.DataFrame()
    box_url = f"https://www.basketball-reference.com{game_link}"
    box_res = requests.get(box_url)
    box_soup = BeautifulSoup(box_res.content, "html.parser")
    tables = box_soup.find_all('table', {'id': lambda x: x and x.endswith('basic')})
    dfs = []
    for table in tables:
        df = pd.read_html(str(table))[0]
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

display(get_box_scores('2024-04-14', 'CLE', 'TOR'))